In [5]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [6]:
# Importamos utilidades y modelos de sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [7]:
df = pd.read_csv('../input/prices/train.csv', parse_dates=['fecha'])
df_test = pd.read_csv('../input/prices/test.csv')
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,nan,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,0.00,4.00,235.00,137.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00


**FEATURE ENGINEERING**

In [8]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0 or metroscubiertos> metrostotales):
        return metroscubiertos
    else:
        return metrostotales

In [9]:
def agregarAnioMesYPrecioDelDolarALaFecha(dfReceived):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

dolar= pd.read_csv('../input/prices/Datos histricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)

In [18]:
propiedades = df.groupby(['tipodepropiedad'])
propiedades = propiedades.agg({
         'habitaciones': "mean",    # Sum duration per group
         'banos': "mean",  # get the count of networks
         'garages': 'mean'  # get the first date per group
    })
propiedades.columns = ["mean_habitacion", "mean_garages", "mean_garages"]
propiedades

,mean_habitacion,mean_garages,mean_garages
tipodepropiedad,,,
Apartamento,2.39,1.82,1.41
Bodega comercial,2.63,1.85,0.48
Casa,3.09,2.24,1.71
Casa en condominio,2.96,2.37,1.91
Casa uso de suelo,4.28,2.40,1.43
Departamento Compartido,2.60,1.79,1.40
Duplex,2.54,1.36,1.18
Edificio,6.13,2.63,0.73
Garage,nan,nan,0.00


In [ ]:
def esNulo(row, feature):
    if(row[feature].isna or row[feature] == 0):
        return True
    else:
        return False

def agregarNulls(row):
    if()

In [ ]:
dfCasas = dfReceived[filtro1]
dfCasas.apply(lambda a: agregarNulls(a))

In [ ]:
def setDataFrameWithDollarPrice(dfReceived, dolar):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month

    #Agrego la columna precio en dolar por propiedad, en el dataframe recibido.
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    #Filtro dejando las cosas útiles.
    #Chequeado de no dejar alguna importante afuera
    dfReceived = dfReceived.filter(["id", "descripcion", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                   'centroscomercialescercanos', 'price', 'precio'])
    print(dfReceived.columns)


    
    #Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
    #tener metros no cubiertos
    filtro1 = dfReceived["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio",
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
    dfCasas = dfReceived[filtro1]
    dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
    dfCasas.garages.fillna(0, inplace=True)

    dfReceived = pd.merge(dfCasas, dfReceived, on=["id", "descripcion", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                  'centroscomercialescercanos', 'price', 'precio'], how='inner')

    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [ ]:
df_train = setDataFrameWithDollarPrice(df, dolar)
df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)

In [ ]:
dolar

In [ ]:
def addColumns(dfReceived):
    dfReceived['metrostotales'].fillna(0, inplace=True)
    dfReceived['metrostotales'] = dfReceived.apply(lambda x: asignarMetros(x['metroscubiertos'], x['metrostotales']),axis=1)
    dfReceived["cant_ambientes"] = dfReceived["habitaciones"] + dfReceived["garages"] + dfReceived["banos"]
    dfReceived["metros_no_cubiertos"] = dfReceived["metrostotales"] - dfReceived["metroscubiertos"]
    dfReceived["precio_dolar_base_2012"] = dolar["price"].min()
    dfReceived["coeficiente_incremento_dolar"] = dfReceived["precio_dolar_base_2012"]/ dfReceived["precioDelDolar"]

In [ ]:
addColumns(df_train)
addColumns(df_test)

df_train

In [ ]:
def addFeature(listaPalabras, nombreDelFeature):
    df_train[nombreDelFeature] = 0
    df_test[nombreDelFeature] = 0
    for palabra in listaPalabras:
        df_train.loc[df_train['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
        df_test.loc[df_test['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
    print("Feature (train) " + nombreDelFeature + ":" + str(df_train[nombreDelFeature].sum()))
    print("Feature (test) " + nombreDelFeature + ":" + str(df_test[nombreDelFeature].sum()))

In [ ]:
addFeature(["luminoso"], "es_luminoso")
addFeature(["amplio"], "es_amplio")
addFeature(["balcon"], "tiene_balcon")
addFeature(["nuevo"], "es_nuevo")
addFeature(["buen estado", "excelente estado", "excelentes condiciones"], "buen_estado")
addFeature(["excelente ubicacion", "buena ubicaion", "ubicacion"], "excelente_ubicacion")
addFeature(["privada", "barrio privado", "privado"], "es_privado")

In [ ]:
def sumarFeatures(dfReceived):
    dfReceived["cant_features"] = dfReceived["es_luminoso"] + dfReceived["es_amplio"] +dfReceived["tiene_balcon"] + \
                                  dfReceived["es_nuevo"] + dfReceived["buen_estado"] + dfReceived["excelente_ubicacion"] + \
                                  dfReceived["es_privado"]

In [ ]:
sumarFeatures(df_train)
sumarFeatures(df_test)

In [ ]:
df_train

In [ ]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols_train = ['descripcion', 'fecha']
drop_cols_test = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha']
df_train = df_train.drop(drop_cols_train, axis=1).copy()

df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)
df_test = df_test.drop(drop_cols_test, axis=1).copy()
print(f"Columnas ({len(df_train.columns)}): {df_train.columns.tolist()}")
df_train.head()

In [ ]:
df_test.dtypes

In [ ]:
display(df_train.isnull().sum())
numeric_columns_with_nulls = list(set(df_train.columns[df_train.isnull().sum() > 0].tolist()) - 
                                  set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

In [ ]:
ciudades_en_train = df_train.ciudad.unique()
ciudades_en_test = df_test.ciudad.unique()

tipodepropiedad_en_train = df_train.tipodepropiedad.unique()
tipodepropiedad_en_test = df_test.tipodepropiedad.unique()

provincia_en_train = df_train.provincia.unique()
provincia_en_test = df_test.provincia.unique()

In [ ]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
df_train_ohe = pd.get_dummies(df_train, dummy_na=True)
print("First")
df_test_ohe = pd.get_dummies(df_test, drop_first=True)

In [ ]:
X = df_train_ohe.drop("precio", axis=1)
y = df_train_ohe['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)


display(df_train_ohe)

In [ ]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
#df2 = pd.get_dummies(df2, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(df_train_ohe.columns)}")
df_train_ohe.head(10)

In [ ]:
numeric_columns_with_nulls = list(set(df_train_ohe.columns[df_train_ohe.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'provincia', 'ciudad']))
print(numeric_columns_with_nulls)

In [ ]:
from sklearn.impute import SimpleImputer
for c in numeric_columns_with_nulls:
    imp = SimpleImputer()
    X_train[c] = imp.fit_transform(X_train[[c]])
    X_test[c] = imp.transform(X_test[[c]])
    df_test_ohe[c] = imp.transform(df_test_ohe[[c]])
    
print ("Finished")

Inserto modelos

In [ ]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(iterations= 1000,
                    learning_rate = 0.06,
                    depth=13,
                    eval_metric='MAE',
                    bagging_temperature = 0.2,
                    metric_period = 75,
                    od_wait=100)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators = 500, 
     learning_rate=0.15, max_depth=10,
     max_features=None, max_leaf_nodes=None,
     min_samples_leaf=3,
     min_samples_split=2, min_weight_fraction_leaf=0.0)

In [ ]:
import xgboost as xgbt
xgb = xgbt.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.10,
                max_depth = 9, min_child_weight = 1, n_estimators = 300, nthread = -1, subsample = 0.7, silent = 1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 100, min_samples_split = 5, max_depth = 15, n_jobs = -1)

In [ ]:
from mlxtend.regressor import StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(cbr, gbr, xgb, rfr),
                                meta_regressor= xgb,
                                use_features_in_secondary=True,
                                cv=2)

Random Forest

In [ ]:
print("Fitting.")

fit_stack = stack_gen.fit(X_train, y_train)

#fit_cbr = cbr.fit(X_train, y_train,
#    eval_set=(X_test,y_test),
#    #cat_features=categorical_features_pos,
#    use_best_model=True,
#   verbose=True)

#print("Catboost regressor: " + str(fit_cbr.score(X_test, y_test)))

#fit_gbr = gbr.fit(X_train, y_train)
#print("Gradient boosting regressor: " + str(gbr.score(X_test, y_test)))

#fit_rfr = rfr.fit(X_train, y_train)
#print("Random forest: " + str(fit_rfr.score(X_test, y_test)))

#fit_xgb = xgb.fit(X_train, y_train)
#print("XGB: " + str(xgb.score(X_test, y_test)))

**BLENDING MODEL**

In [ ]:
fit_stack.score(np.array(X_test), np.array(y_test))

In [ ]:
def blending(X):
    return  fit_stack.predict(X)

In [ ]:
for ciudad in ciudades_en_train:
    if ciudad not in ciudades_en_test:
        df_test_ohe['ciudad_'+ str(ciudad)] = 0
        
for provincia in provincia_en_train:
    if provincia not in provincia_en_test:
        df_test_ohe['provincia_'+ str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        df_test_ohe['tipodepropiedad_'+ str(tipodepropiedad)] = 0

In [ ]:
some_list = ['tipodepropiedad_Apartamento', 'ciudad_Abasolo', 'provincia_Aguascalientes', 'tipodepropiedad_nan']
for item in some_list:
    df_test_ohe[item] = 0

In [ ]:
#df_test_ohe.drop("price_y", axis = "columns",inplace = True)
df_test_ohe.rename(columns={"price_x": "precioDelDolar"}, inplace = True)
df_test_ohe

In [ ]:
df_test_ohe = df_test_ohe[list(X_train.columns.values)]

In [ ]:
y_test

In [ ]:
y_false_pred = blending(np.array(X_test))


In [ ]:
y_false_pred

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_false_pred))

In [ ]:
df_test_ohe = df_test_ohe.loc[:, ~df_test_ohe.columns.duplicated()]
df_test_ohe

In [ ]:
nulls = list(set(df_test_ohe.columns[df_test_ohe.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'provincia', 'ciudad']))
print(nulls)

In [ ]:
from sklearn.impute import SimpleImputer
obj = SimpleImputer(strategy='mean')
for c in nulls:
    df_test_ohe[c] = imp.fit_transform(df_test_ohe[[c]])
    
print ("Finished")

In [ ]:
df_test_ohe

In [ ]:
#Cosas para entregar
X_real_test = df_test_ohe
y_real_pred = blending(np.array(X_real_test))
y_real_pred

In [ ]:
res = pd.DataFrame(y_real_pred, index= df_test_ohe.index, columns=['precio'])
res2 = res
res2['id'] = df_test_ohe["id"]
res = res.reset_index(drop=True)
res.set_index('id', inplace = True)
res.rename(columns={"precio": "target"}, inplace = True)

In [ ]:
res

In [ ]:
res.to_csv("stacked1.csv", header=True)